In [10]:
# import relevant libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE

In [2]:
# load preprocessed data
X_train = pd.read_csv(r'/Users/adityamxr/Desktop/spotify-time-series/data-analysis/X_train_scaled.csv')
X_test = pd.read_csv(r'/Users/adityamxr/Desktop/spotify-time-series/data-analysis/X_test_scaled.csv')
y_train = pd.read_csv(r'/Users/adityamxr/Desktop/spotify-time-series/data-analysis/y_train.csv')
y_test = pd.read_csv(r'/Users/adityamxr/Desktop/spotify-time-series/data-analysis/y_test.csv')

In [3]:
# verify import

X_train.head()

,acousticness,danceability,energy,explicit,instrumentalness,liveness,loudness,mode,speechiness,tempo,valence,labelenc_genre,duration_s
0,-1.306496,-1.876611,1.091134,0,-0.441361,0.031520,1.435897,1,-0.285366,-0.456439,0.358882,216,-0.765607
1,0.994996,-0.804675,1.072424,0,-0.521536,2.231258,1.467179,0,0.261131,-1.118284,1.308006,95,-0.458954
2,-0.244792,0.803230,-0.065093,0,-0.523093,1.954170,-0.225053,1,-0.241940,-0.128433,0.541846,403,-0.680081
3,-0.887251,0.974284,0.619663,0,-0.523104,-0.596169,0.965960,1,-0.445708,-0.258020,1.132666,403,-0.315422
4,-1.200251,0.153226,0.522375,0,-0.523153,-0.735844,0.152982,1,-0.349503,0.342224,1.388053,403,0.095724


In [4]:
# verify import

X_test.head()

,acousticness,danceability,energy,explicit,instrumentalness,liveness,loudness,mode,speechiness,tempo,valence,labelenc_genre,duration_s
0,-1.035920,1.487445,-0.802234,0,-0.523051,-0.623878,-1.197975,1,-0.379567,-0.259388,-0.007046,519,-0.410379
1,1.284368,0.626475,-1.681191,0,-0.522972,-0.648759,-1.945562,1,-0.378899,-1.325988,-0.780830,488,0.078591
2,1.300297,0.780423,-0.457985,0,-0.523153,3.594078,0.689371,1,0.107471,-0.662287,0.515163,500,-1.108589
3,1.273749,-1.397661,-1.266221,0,-0.519629,-0.409559,-0.266939,0,-0.360192,-1.758596,-1.642285,344,-0.154872
4,0.721552,0.124717,-0.405600,0,1.772205,-0.806529,-0.069350,1,-0.396269,-1.178940,-0.544502,224,3.124084


In [5]:
# verify import

y_train.head()

,popularity_binned
0,Medium
1,Very Low
2,Low
3,Medium
4,Low


In [6]:
# verify import

y_test.head()

,popularity_binned
0,Medium
1,Low
2,Very Low
3,Low
4,Medium


In [7]:
# ensure target variables are in the correct format
y_train = y_train.values.ravel()  # Convert to 1D array if needed
y_test = y_test.values.ravel()

# verify shapes
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (135927, 13)
X_test shape: (33982, 13)
y_train shape: (135927,)
y_test shape: (33982,)


In [9]:
# compute the cross-val score before gridsearch rf to get baseline for comparison

# initialize
rf_clf = RandomForestClassifier(random_state=42)

# 3-fold cv
cv_scores = cross_val_score(rf_clf, X_train, y_train, cv=3, scoring='accuracy')
print(f"Mean Cross-Validation Score: {cv_scores.mean():.4f}")

Mean Cross-Validation Score: 0.5861


In [11]:
# mean CV score is too low, add SMOTE